In [1]:
import random
import numpy as np
import aifc
import csv
import pandas as pd
import sys
import matplotlib.pyplot as plt

def extract_file():
    df = pd.read_csv('train.csv', header=0, sep=',')
    df.columns=['sample', 'label'] 
    return df
    
def pourc(tl, pourc):
    return pourc*tl/100

def compare_our_arrays(x,y):
    score = 0
    for i in x: 
        if x[i] == y[i]:
            score+=1
    return score

def create_label():
    df = extract_file()
    label_list = []
    for i in range(len(df)):
        label_list.append(df['label'][i])
    return label_list

def split_our_lists(): #Splits our lists 
    train = []
    validation = []
    test = []
    label_list = create_label()
    random.shuffle(label_list)
    tl = len(label_list)
    
    for i in range(tl):
        if i < pourc(tl,60):
            train.append(label_list[i])
        elif i < pourc(tl,80):
            validation.append(label_list[i])
        else:
            test.append(label_list[i])
    return train, validation, test


def show_signal(filename): #shows our signal thx to matplotlib
    liste_sample_fft = []
    s = aifc.open('Data/train/' + filename, "r")
    N = s.getnframes()
    strsig = s.readframes(N)
    y = np.fromstring(strsig, np.short).byteswap()
    FE = s.getframerate()
    A = np.fft.fft(y)
    DeltaT = 1./FE
    ampl = np.abs(A)*1./N
    freqs = np.fft.fftfreq(N, DeltaT)
    P = plt.plot(freqs[:int(N/2)], ampl[:int(N/2)]) 
    plt.ylabel("Amplitude")
    plt.xlabel("Frequence")
    plt.show()

def fct_FFT(filename) : #Fast Fourier transformation decomposes our temporal signal into frequency signal
    liste_sample_fft = []
    s = aifc.open('Data/train/' + filename, "r")
    N = s.getnframes()
    strsig = s.readframes(N)
    y = np.fromstring(strsig, np.short).byteswap()
    FE = s.getframerate()
    A = np.fft.fft(y)
    DeltaT = 1./FE
    ampl = np.abs(A)*1./N
    liste_sample_fft.append(ampl)
    return ampl

def Parcours_file_fft(listname): #Allows us to take the name of the files of train or test or validation
    df = extract_file() 
    df_train = []
    for i in range(len(listname)):
        df_train.append(fct_FFT(df["sample"][i]))
    return df_train

liste = split_our_lists() 

Train = liste[0]
Train_fft = Parcours_file_fft(Train)

Validation = liste[1]
Validation_fft = Parcours_file_fft(Validation)

Test = liste[2]
Test_fft = Parcours_file_fft(Test)


In [26]:
# Zone test hyperparamètres
liste_de_test = []
for i in [2,5,7,30,50,100]:
    ClassifiedDatas = RandomForestClassifier(n_estimators = 7, max_depth = 1, min_samples_split = 5)#pas un problème multidimensionnel (on prend que la fréquence)
    ClassifiedDatas.fit(Train_fft, Train)
    P = ClassifiedDatas.score(Test_fft, Test)
    liste_de_test.append(P)
    print('pour n_estim =',i,':',P)


pour n_estim = 2 : 0.762
pour n_estim = 5 : 0.762
pour n_estim = 7 : 0.762
pour n_estim = 30 : 0.762
pour n_estim = 50 : 0.762
pour n_estim = 100 : 0.762


In [37]:
def optimal_method(input_fft,input_s, size = 0):
    ClassifiedDatas = RandomForestClassifier(n_estimators = 7, max_depth = 1, min_samples_split = 5)
    if size == 0:
        ClassifiedDatas.fit(Train_fft, Train)
        return ClassifiedDatas.score(input_fft, input_s)
    else :
        ClassifiedDatas.fit(Train_fft[:size], Train[:size])
        return ClassifiedDatas.score(input_fft[:size], input_s[:size])
    
optimal_method(Validation_fft,Validation)

0.77316666666666667

In [17]:
print(ClassifiedDatas.get_params(deep = True))


{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 1, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_split': 1e-07, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 7, 'n_jobs': 1, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [38]:
# zone test nearest_neighbor
from sklearn.neighbors import NearestNeighbors
k = 5
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(Train_fft[:k])
distances, indices = nbrs.kneighbors(Train_fft[:k])
print(distances & indices)

TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

(<15x1418 sparse matrix of type '<class 'numpy.int64'>'
	with 1720 stored elements in Compressed Sparse Row format>, array([   0,  155,  314,  455,  594,  733,  872, 1015, 1154, 1285, 1418]))
